In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import cv2
import numpy as np
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report

In [ ]:
!git clone https://github.com/jahongirkb/Echocardiography.git

Cloning into 'Echocardiography'...
remote: Enumerating objects: 8331, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 8331 (delta 0), reused 3 (delta 0), pack-reused 8328
Receiving objects: 100% (8331/8331), 488.02 MiB | 33.39 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Checking out files: 100% (8328/8328), done.


In [ ]:
 TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data():
  size = 224
  # raw
  transform_train_raw = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor()
  ])

  # preprocess
  transform_train_preprocess = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.GaussianBlur(3),
    transforms.RandomRotation(2.8),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.ToTensor()
  ])

  # augmentation
  transform_train_augmentation = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.RandomCrop(size, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.1),
    transforms.ToTensor()
  ])

  # test
  transform_test = transforms.Compose([
    transforms.Resize((size, size)),                                 
    transforms.ToTensor()
  ])

  trainset = torchvision.datasets.ImageFolder(
      root='/content/Echocardiography/DATA_CHAMBER_2021/train', transform=transform_train_augmentation)
  testset = torchvision.datasets.ImageFolder(
      root='/content/Echocardiography/DATA_CHAMBER_2021/test', transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  batch_size = 32
  trainloader = DataLoader(dataset=datasets.train, batch_size=batch_size, shuffle=True, num_workers=2)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch_size, shuffle=False, num_workers=2)
  return TrainTest(train=trainloader, test=testloader)


In [ ]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_step = 40
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    
    outputs = model(images)
    loss = loss_func(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    running_loss += loss.item()
    if i % reporting_step == reporting_step - 1:
        print(f"Epoch {epoch} Step {i} ave_loss {running_loss/reporting_step:0.4f}")
        running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  pred = []
  true = []
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      pred += list(predicted.cpu().numpy())
      true += list(labels.cpu().numpy())
  return pred, true

In [ ]:
def main(model=None):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print(device)

  if model == 'vgg16':
    PATH = './vgg16.pth'
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  elif model == 'vgg19':
    PATH = './vgg19.pth'
    model = torchvision.models.vgg19()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  elif model == 'resnet50':
    PATH = './resnet50.pth'
    model = torchvision.models.resnet50()
    model.fc = torch.nn.modules.Linear(in_features=2048, out_features=3)

  model.to(device)
  loss_function = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_function, optimizer, device)
    pred, true = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(true, pred, target_names=classes))  
    print("---------------------------------------------------------------")      

    torch.save(model.state_dict(), PATH)
  return model

In [ ]:
modelVGG16 = main(model='vgg16')

cuda:0
Epoch 0 Step 39 ave_loss 1.0945
Epoch 0 Step 79 ave_loss 1.0941
Epoch 0 Step 119 ave_loss 1.0854
Epoch 0 Step 159 ave_loss 1.0595
Epoch 0 Step 199 ave_loss 1.0100


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          2C       0.00      0.00      0.00       409
          3C       0.00      0.00      0.00       367
          4C       0.52      1.00      0.68       831

    accuracy                           0.52      1607
   macro avg       0.17      0.33      0.23      1607
weighted avg       0.27      0.52      0.35      1607

---------------------------------------------------------------
Epoch 1 Step 39 ave_loss 1.0408
Epoch 1 Step 79 ave_loss 1.0452
Epoch 1 Step 119 ave_loss 0.9253
Epoch 1 Step 159 ave_loss 0.7413
Epoch 1 Step 199 ave_loss 0.6858
              precision    recall  f1-score   support

          2C       0.85      0.56      0.68       409
          3C       0.38      0.89      0.53       367
          4C       0.98      0.57      0.72       831

    accuracy                           0.64      1607
   macro avg       0.73      0.67      0.64      1607
weighted avg       0.81      0.64      0.66      1607

-----------

In [ ]:
modelVGG19 = main(model='vgg19')

cuda:0
Epoch 0 Step 39 ave_loss 1.0974
Epoch 0 Step 79 ave_loss 1.0920
Epoch 0 Step 119 ave_loss 1.0964
Epoch 0 Step 159 ave_loss 1.0973
Epoch 0 Step 199 ave_loss 1.0956


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          2C       0.25      1.00      0.41       409
          3C       0.00      0.00      0.00       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.25      1607
   macro avg       0.08      0.33      0.14      1607
weighted avg       0.06      0.25      0.10      1607

---------------------------------------------------------------
Epoch 1 Step 39 ave_loss 1.0978
Epoch 1 Step 79 ave_loss 1.0958
Epoch 1 Step 119 ave_loss 1.0986
Epoch 1 Step 159 ave_loss 1.1000
Epoch 1 Step 199 ave_loss 1.0962


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          2C       0.25      1.00      0.41       409
          3C       0.00      0.00      0.00       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.25      1607
   macro avg       0.08      0.33      0.14      1607
weighted avg       0.06      0.25      0.10      1607

---------------------------------------------------------------
Epoch 2 Step 39 ave_loss 1.0961
Epoch 2 Step 79 ave_loss 1.0960
Epoch 2 Step 119 ave_loss 1.0991
Epoch 2 Step 159 ave_loss 1.0980
Epoch 2 Step 199 ave_loss 1.0983


...

              precision    recall  f1-score   support

          2C       0.00      0.00      0.00       409
          3C       0.23      1.00      0.37       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.23      1607
   macro avg       0.08      0.33      0.12      1607
weighted avg       0.05      0.23      0.08      1607

---------------------------------------------------------------
Epoch 8 Step 39 ave_loss 1.0966
Epoch 8 Step 79 ave_loss 1.0979
Epoch 8 Step 119 ave_loss 1.0976
Epoch 8 Step 159 ave_loss 1.0991
Epoch 8 Step 199 ave_loss 1.0951


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          2C       0.25      1.00      0.41       409
          3C       0.00      0.00      0.00       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.25      1607
   macro avg       0.08      0.33      0.14      1607
weighted avg       0.06      0.25      0.10      1607

---------------------------------------------------------------
Epoch 9 Step 39 ave_loss 1.0982
Epoch 9 Step 79 ave_loss 1.0981
Epoch 9 Step 119 ave_loss 1.0975
Epoch 9 Step 159 ave_loss 1.0973
Epoch 9 Step 199 ave_loss 1.0943


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          2C       0.00      0.00      0.00       409
          3C       0.23      1.00      0.37       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.23      1607
   macro avg       0.08      0.33      0.12      1607
weighted avg       0.05      0.23      0.08      1607

---------------------------------------------------------------


In [ ]:
modelResNet50 = main(model='resnet50')

cuda:0
Epoch 0 Step 39 ave_loss 3.5496
Epoch 0 Step 79 ave_loss 1.5851
Epoch 0 Step 119 ave_loss 1.3564
Epoch 0 Step 159 ave_loss 1.1872
Epoch 0 Step 199 ave_loss 1.1425
              precision    recall  f1-score   support

          2C       0.26      0.90      0.40       409
          3C       0.31      0.14      0.20       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.26      1607
   macro avg       0.19      0.35      0.20      1607
weighted avg       0.14      0.26      0.15      1607

---------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1 Step 39 ave_loss 1.0797
Epoch 1 Step 79 ave_loss 1.0303
Epoch 1 Step 119 ave_loss 0.9923
Epoch 1 Step 159 ave_loss 0.9309
Epoch 1 Step 199 ave_loss 0.8660


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          2C       0.37      0.90      0.52       409
          3C       0.21      0.36      0.27       367
          4C       0.00      0.00      0.00       831

    accuracy                           0.31      1607
   macro avg       0.19      0.42      0.26      1607
weighted avg       0.14      0.31      0.19      1607

---------------------------------------------------------------
Epoch 2 Step 39 ave_loss 0.8437
Epoch 2 Step 79 ave_loss 0.6863
Epoch 2 Step 119 ave_loss 0.7407
Epoch 2 Step 159 ave_loss 0.5750
Epoch 2 Step 199 ave_loss 0.4624
              precision    recall  f1-score   support

          2C       0.66      0.67      0.67       409
          3C       0.56      0.62      0.59       367
          4C       0.87      0.82      0.84       831

    accuracy                           0.74      1607
   macro avg       0.70      0.70      0.70      1607
weighted avg       0.74      0.74      0.74      1607

-----------